<a href="https://colab.research.google.com/github/NishatVasker/Machine_learning_projects/blob/main/Car_Damage_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# install dependencies
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git' 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__
    
     !git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo
     

In [ ]:
import os
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import itertools
import json
import random

from detectron2.structures import BoxMode
from detectron2.utils.visualizer import Visualizer
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog

In [ ]:

def getImageDicts(imgDir):
    jsonFile = os.path.join(imgDir,"via_region_data.json")
    
    with open(jsonFile) as f:
        imgAnnotations = json.load(f)
    datasetDicts = []
    for id, value in enumerate(imgAnnotations.values()):
      
        record= {}
      
        filepath = os.path.join(imgDir,value['filename'])
        try:
            height, width = cv2.imread(filepath).shape[:2]
        except AttributeError:
            continue
        record['file_name'] = filepath
        record['image_id'] = id
        record['height'] = height
        record['width'] = width

        annos = value['regions']
        objs = []
        for _, anno in annos.items():
            anno = anno['shape_attributes']
            px = anno['all_points_x']
            py = anno['all_points_y']
            poly = [(x + 0.5, y + 0.5) for x,y in zip(px,py)]
            poly = list(itertools.chain.from_iterable(poly))

            obj = {
                'bbox' : [np.min(px), np.min(py), np.max(px), np.max(py)],
                'bbox_mode' : BoxMode.XYXY_ABS,
                'segmentation' : [poly],
                'category_id' : 0,
                'iscrowd' : 0
            }
            objs.append(obj)
        record['annotations'] = objs
        datasetDicts.append(record)
    print("HERE")
    return datasetDicts
     
datasetPath = "/content/drive/My Drive/Car-Damage-Dataset/"
for d in ["train","val"]:
    #DatasetCatalog.clear()
    DatasetCatalog.register("car_dataset_"+d, lambda d=d: getImageDicts(datasetPath + d))
    MetadataCatalog.get("car_dataset_"+d).thing_classes = ["damage"]
     

print(len(imageDicts))

In [ ]:

imageDicts = getImageDicts(datasetPath+"train")
metadata = MetadataCatalog.get("car_dataset_train")
for item in random.sample(imageDicts, 3):
    img = cv2.imread(item['file_name'])
    visualizer = Visualizer(img[:,:,::-1], metadata=metadata,scale=0.5)
    vis = visualizer.draw_dataset_dict(item)
    cv2_imshow(vis.get_image()[:,:,::-1])

In [ ]:
#training configuration
cfg = get_cfg()
cfg.merge_from_file("./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("car_dataset_train", )
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.002
cfg.SOLVER.MAX_ITER = 350   
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()
     

In [ ]:

%reload_ext tensorboard
%tensorboard --logdir output
     

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
#cfg.MODEL.WEIGHTS = "/content/drive/My Drive/Saved Models/BestModels/Maskrcnn/model_final_detectron2_44.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
cfg.DATASETS.TEST = ("car_dataset_val", )
predictor = DefaultPredictor(cfg)
     

In [ ]:
from detectron2.utils.visualizer import ColorMode
valImgDicts = getImageDicts(datasetPath+"val")
for d in random.sample(valImgDicts, 10):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    #print(outputs)
    v = Visualizer(im[:, :, ::-1],
                   metadata=metadata, 
                   scale=0.5,
                   instance_mode=ColorMode.IMAGE_BW 
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("car_dataset_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "car_dataset_val")
inference_on_dataset(trainer.model, val_loader, evaluator)
# another equivalent way is to use trainer.test